In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

# Set up Chrome config
options = webdriver.ChromeOptions()
# Line that hides the browser window
options.add_argument('--headless')
# prevent GPU-related crashes
options.add_argument('--disable-gpu')
# Launch Chrome with options
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# define url that we want to scrape
url = "https://fbref.com/en/comps/9/Premier-League-Stats"
driver.get(url)

# to avoid rate limit, don't want IP to get jailed
time.sleep(10)

# save the page content before quitting driver
html_text = driver.page_source
driver.quit()  # can't access driver after this


# Parse page with BeautifulSoup
soup = BeautifulSoup(html_text, 'html.parser')
# Find all stats tables
tables = soup.select('table.stats_table')
print(f"Found {len(tables)} stats tables")



Found 25 stats tables


In [3]:
html_text[:1000]

'<html data-version="klecko-" data-root="/home/fb/deploy/www/base" lang="en" class=" js cookies localstorage sessionstorage cors history csspositionsticky no-touchevents pointerevents matchmedia flexwrap desktop is_live" style=""><head><script src="https://rules.quantcount.com/rules-p-UeXruRVtZz7w6.js" async=""></script><script async="" src="//c.amazon-adsystem.com/aax2/apstag.js"></script><script src="https://btloader.com/tag?o=5714937848528896&amp;upapi=true"></script><style><!----> <!--?lit$775861955$-->.osano-cm-window{font-family:Helvetica,Arial,Hiragino Sans GB,STXihei,Microsoft YaHei,WenQuanYi Micro Hei,Hind,MS Gothic,Apple SD Gothic Neo,NanumBarunGothic,sans-serif;font-size:16px;font-smooth:always;-webkit-font-smoothing:antialiased;-moz-osx-font-smoothingz:auto;display:block;left:0;line-height:1;position:absolute;top:0;width:100%;z-index:2147483638;--fade-transition-time:700ms;--slide-transition-time:400ms}.osano-cm-window--context_amp{height:100%}.osano-visually-hidden{height:

In [107]:
# Goal: retrieve/store the "Premier League" overall standings table in a pandas dataframe
import pandas as pd
from io import StringIO

html_content = StringIO(html_text)
# match looks for a specific string inside the table, pd.read_html reads all the tables on the page, turns the selected table into a pandas dataframe
squad_standings = pd.read_html(html_content, match="Premier League")[0]
# sort the team in order by team name, so that it aligns with the "Squad Standard Stats" table
squad_standings_sorted = squad_standings.sort_values(by="Squad", ascending=True)
squad_standings_sorted.head()

,Rk,Squad,MP,W,D,L,GF,GA,GD,Pts,Pts/MP,xG,xGA,xGD,xGD/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes
1,2,Arsenal,36,18,14,4,66,33,33,68,1.89,57.0,32.3,24.7,0.69,D W D L D,60256,Kai Havertz - 9,David Raya,NaN
5,6,Aston Villa,36,18,9,9,56,49,7,63,1.75,54.3,46.7,7.6,0.21,W W L W W,42070,Ollie Watkins - 16,Emiliano Martínez,NaN
9,10,Bournemouth,36,14,11,11,55,43,12,53,1.47,61.3,46.7,14.6,0.41,W D D W L,10587,Justin Kluivert - 12,Kepa Arrizabalaga,NaN
7,8,Brentford,36,16,7,13,63,53,10,55,1.53,55.0,53.4,1.6,0.05,D W W W W,18832,"Yoane Wissa, Bryan Mbeumo - 18",Mark Flekken,NaN
8,9,Brighton,36,14,13,9,59,56,3,55,1.53,54.3,50.3,4.0,0.11,D L W D W,31808,"Danny Welbeck, João Pedro - 10",Bart Verbruggen,NaN


In [108]:
# exclude rank, GF and GA (because Goal Difference (GD) represents them combined), Pts, Pts/MP, Last 5 (not relevant to whole season)
# xG and xGA (because Expected Goal Difference (xGD) represents them combined), xGD/90, Top Team Scorer, Goalkeeper, Notes
squad_standings_sorted.columns

Index(['Rk', 'Squad', 'MP', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'Pts', 'Pts/MP',
       'xG', 'xGA', 'xGD', 'xGD/90', 'Last 5', 'Attendance', 'Top Team Scorer',
       'Goalkeeper', 'Notes'],
      dtype='object')

In [109]:
# Squad standings sorted columns to grab: Squad, W, D, L, GD, xGD, Attendance (per game)
squad_standings_filtered = squad_standings_sorted[['Squad', 'W', 'D', 'L', 'GD', 'Attendance']]
squad_standings_filtered.head()

,Squad,W,D,L,GD,Attendance
1,Arsenal,18,14,4,33,60256
5,Aston Villa,18,9,9,7,42070
9,Bournemouth,14,11,11,12,10587
7,Brentford,16,7,13,10,18832
8,Brighton,14,13,9,3,31808


In [110]:
squad_standings_filtered.columns

Index(['Squad', 'W', 'D', 'L', 'GD', 'Attendance'], dtype='object')

In [111]:
# Goal: retrieve/store the "Squad Standard Stats" table in a pandas dataframe

# match looks for a specific string inside the table, specifically "Squad Standard Stats"
squad_stats = pd.read_html(html_content, match="Squad Standard Stats")[0]
# drops the top level index
squad_stats.columns = squad_stats.columns.droplevel()
squad_stats.head()

,Squad,# Pl,Age,Poss,MP,Starts,Min,90s,Gls,Ast,...,Gls,Ast,G+A,G-PK,G+A-PK,xG,xAG,xG+xAG,npxG,npxG+xAG
0,Arsenal,25,26.6,56.9,36,396,3240,36.0,64,52,...,1.78,1.44,3.22,1.72,3.17,1.58,1.18,2.77,1.54,2.72
1,Aston Villa,28,27.8,50.6,36,396,3240,36.0,54,43,...,1.50,1.19,2.69,1.42,2.61,1.51,1.12,2.63,1.38,2.50
2,Bournemouth,29,25.9,48.3,36,396,3240,36.0,54,39,...,1.50,1.08,2.58,1.33,2.42,1.70,1.16,2.87,1.55,2.71
3,Brentford,28,26.6,47.7,36,396,3240,36.0,62,42,...,1.72,1.17,2.89,1.58,2.75,1.53,1.12,2.65,1.42,2.54
4,Brighton,31,25.7,52.1,36,396,3240,36.0,57,38,...,1.58,1.06,2.64,1.42,2.47,1.51,1.05,2.56,1.38,2.43


In [112]:
# exclude MP, Starts, Min, 90s, Gls and Ast (because Goals + Assits (G+A) represents them combined)
# PKatt (penalty kick attempts, already have penalty kick goals), xG (Epected goals we already have)
# npxG and xAG (because npxG + xAG combined them both), Per 90 minute columns (Gls - npxG+xAG -> redundant)
squad_stats.columns

Index(['Squad', '# Pl', 'Age', 'Poss', 'MP', 'Starts', 'Min', '90s', 'Gls',
       'Ast', 'G+A', 'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR', 'xG', 'npxG',
       'xAG', 'npxG+xAG', 'PrgC', 'PrgP', 'Gls', 'Ast', 'G+A', 'G-PK',
       'G+A-PK', 'xG', 'xAG', 'xG+xAG', 'npxG', 'npxG+xAG'],
      dtype='object')

In [113]:
# Find all indices of columns named "Gls" (Goals columns)
gls_indices = [i for i, col in enumerate(squad_stats.columns) if col == "Gls"]
# Get index of the second "Gls" (after "PrgP" column)
second_gls_index = gls_indices[1]
# filter for only columns before that
squad_stats = squad_stats.iloc[:, :second_gls_index]
squad_stats.columns



Index(['Squad', '# Pl', 'Age', 'Poss', 'MP', 'Starts', 'Min', '90s', 'Gls',
       'Ast', 'G+A', 'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR', 'xG', 'npxG',
       'xAG', 'npxG+xAG', 'PrgC', 'PrgP'],
      dtype='object')

In [114]:
# Squad Standard Stats columns to grab: Squad, # Pl (# of players), Age, Poss (% passess attempted), G+A (Goals + Assists)
# CrdY (Yellow Cards), CrdR (Red Cards), PrgC (Progressive Carries), PrgP (Progressive passes)
squad_stats_filtered = squad_stats[['Squad', '# Pl', 'Age', 'Poss', 'G+A', 'CrdY', 'CrdR', 'PrgC', 'PrgP']]
squad_stats_filtered.head()



,Squad,# Pl,Age,Poss,G+A,CrdY,CrdR,PrgC,PrgP
0,Arsenal,25,26.6,56.9,116,67,6,806,1738
1,Aston Villa,28,27.8,50.6,97,73,3,678,1254
2,Bournemouth,29,25.9,48.3,93,94,2,719,1411
3,Brentford,28,26.6,47.7,104,57,1,559,1273
4,Brighton,31,25.7,52.1,95,76,3,766,1432


In [115]:
squad_stats_filtered.columns

Index(['Squad', '# Pl', 'Age', 'Poss', 'G+A', 'CrdY', 'CrdR', 'PrgC', 'PrgP'], dtype='object')

In [116]:
# retrieving the "Squad Shooting" table and storing it into a pandas dataframe
squad_shooting = pd.read_html(html_content, match="Squad Shooting")[0]
# drops the top level index
squad_shooting.columns = squad_shooting.columns.droplevel()
squad_shooting.head()

,Squad,# Pl,90s,Gls,Sh,SoT,SoT%,Sh/90,SoT/90,G/Sh,G/SoT,Dist,FK,PK,PKatt,xG,npxG,npxG/Sh,G-xG,np:G-xG
0,Arsenal,25,36.0,64,509,167,32.8,14.14,4.64,0.12,0.37,15.0,10,2,2,57.0,55.5,0.11,7.0,6.5
1,Aston Villa,28,36.0,54,454,151,33.3,12.61,4.19,0.11,0.34,16.2,12,3,6,54.3,49.5,0.11,-0.3,1.5
2,Bournemouth,29,36.0,54,546,183,33.5,15.17,5.08,0.09,0.26,16.7,9,6,7,61.3,55.8,0.10,-7.3,-7.8
3,Brentford,28,36.0,62,411,162,39.4,11.42,4.50,0.14,0.35,15.0,10,5,5,55.0,51.2,0.13,7.0,5.8
4,Brighton,31,36.0,57,479,164,34.2,13.31,4.56,0.11,0.31,17.2,24,6,6,54.3,49.6,0.11,2.7,1.4


In [117]:
squad_shooting.columns

Index(['Squad', '# Pl', '90s', 'Gls', 'Sh', 'SoT', 'SoT%', 'Sh/90', 'SoT/90',
       'G/Sh', 'G/SoT', 'Dist', 'FK', 'PK', 'PKatt', 'xG', 'npxG', 'npxG/Sh',
       'G-xG', 'np:G-xG'],
      dtype='object')

In [118]:
# Squad shooting columns to grab: "Squad", "Sh" (Shots Total), SoT (Shots on Target), G/Sh (Goals per Shot), Dist (Average Shot Distance)
squad_shooting_filtered = squad_shooting[['Squad', 'Sh', 'SoT', 'G/Sh', 'Dist']]
squad_shooting_filtered.head()

,Squad,Sh,SoT,G/Sh,Dist
0,Arsenal,509,167,0.12,15.0
1,Aston Villa,454,151,0.11,16.2
2,Bournemouth,546,183,0.09,16.7
3,Brentford,411,162,0.14,15.0
4,Brighton,479,164,0.11,17.2


In [119]:
squad_shooting_filtered.columns

Index(['Squad', 'Sh', 'SoT', 'G/Sh', 'Dist'], dtype='object')

In [120]:
# retrieving the "Squad Passing" table and storing it into a pandas dataframe
squad_passing = pd.read_html(html_content, match="Squad Passing")[0]

# combine the top-level headers(Total, Short, Medium, Long) with their subcolumns
squad_passing.columns = [' '.join(col).strip() if col[0] != 'Unnamed' else col[1] for col in squad_passing.columns.values]
# gets rid of the "Unamed: " column prefix portions
squad_passing.columns = [col.split(' ', 1)[-1] if col.startswith('Unnamed') else col for col in squad_passing.columns]
# gets rid of "_level_0" column prefix portions
squad_passing.columns = [
    col.split(' ', 1)[-1] if '_level_0' in col else col
    for col in squad_passing.columns
]

squad_passing.head()

,Squad,# Pl,90s,Total Cmp,Total Att,Total Cmp%,Total TotDist,Total PrgDist,Short Cmp,Short Att,...,Long Cmp%,Ast,xAG,Expected xA,Expected A-xAG,KP,1/3,PPA,CrsPA,PrgP
0,Arsenal,25,36.0,16194,19213,84.3,265139,83064,8127,8858,...,51.3,52,42.6,42.9,9.4,396,1307,414,68,1738
1,Aston Villa,28,36.0,13936,16849,82.7,239534,83669,6413,7109,...,53.1,43,40.4,35.1,2.6,342,1078,297,54,1254
2,Bournemouth,29,36.0,12136,15871,76.5,219671,82330,5249,6076,...,48.2,39,41.9,34.2,-2.9,419,1175,314,96,1411
3,Brentford,28,36.0,12643,16194,78.1,231997,88236,5542,6393,...,52.6,42,40.4,36.7,1.6,315,1039,304,85,1273
4,Brighton,31,36.0,14901,18107,82.3,253242,85792,7235,7998,...,54.4,38,37.8,35.5,0.2,342,1107,314,84,1432


In [121]:
squad_passing.columns

Index(['Squad', '# Pl', '90s', 'Total Cmp', 'Total Att', 'Total Cmp%',
       'Total TotDist', 'Total PrgDist', 'Short Cmp', 'Short Att',
       'Short Cmp%', 'Medium Cmp', 'Medium Att', 'Medium Cmp%', 'Long Cmp',
       'Long Att', 'Long Cmp%', 'Ast', 'xAG', 'Expected xA', 'Expected A-xAG',
       'KP', '1/3', 'PPA', 'CrsPA', 'PrgP'],
      dtype='object')

In [122]:
# Squad passing columns to grab: "Squad", "Total Cmp%" (Total Pass Completion %), "Short Cmp%" (Short Pass Completion %)
# "Medium Cmp%" (Medium Pass Completion %), "Long Cmp%" (Long Pass Completion %)
squad_passing_filtered = squad_passing[['Squad', 'Total Cmp%', 'Short Cmp%', 'Medium Cmp%', 'Long Cmp%']]
squad_passing_filtered.head()

,Squad,Total Cmp%,Short Cmp%,Medium Cmp%,Long Cmp%
0,Arsenal,84.3,91.7,89.1,51.3
1,Aston Villa,82.7,90.2,88.9,53.1
2,Bournemouth,76.5,86.4,84.6,48.2
3,Brentford,78.1,86.7,84.8,52.6
4,Brighton,82.3,90.5,87.4,54.4


In [123]:
squad_passing_filtered.columns

Index(['Squad', 'Total Cmp%', 'Short Cmp%', 'Medium Cmp%', 'Long Cmp%'], dtype='object')

In [124]:
# retrieving the "Squad Goal and Shot Creation" table and storing it into a pandas dataframe
squad_goal_shot_creation = pd.read_html(html_content, match="Squad Goal and Shot Creation")[0]
# drops the top level index
squad_goal_shot_creation.columns = squad_goal_shot_creation.columns.droplevel()
squad_goal_shot_creation.head()

,Squad,# Pl,90s,SCA,SCA90,PassLive,PassDead,TO,Sh,Fld,Def,GCA,GCA90,PassLive,PassDead,TO,Sh,Fld,Def
0,Arsenal,25,36.0,903,25.08,677,78,46,57,34,11,113,3.14,81,11,5,11,4,1
1,Aston Villa,28,36.0,812,22.56,574,82,48,56,38,14,98,2.72,77,6,3,6,4,2
2,Bournemouth,29,36.0,964,26.78,735,65,40,65,38,21,91,2.53,60,6,5,8,10,2
3,Brentford,28,36.0,743,20.64,542,70,39,47,34,11,108,3.00,75,11,5,7,9,1
4,Brighton,31,36.0,855,23.75,633,62,52,43,51,14,101,2.81,71,6,9,4,9,2


In [125]:
# Find the index of the "GCA" column
gca_index = squad_goal_shot_creation.columns.get_loc("GCA")
# Keep only columns to the left of "GCA"
squad_goal_shot_creation = squad_goal_shot_creation.iloc[:, :gca_index]

squad_goal_shot_creation.head()

,Squad,# Pl,90s,SCA,SCA90,PassLive,PassDead,TO,Sh,Fld,Def
0,Arsenal,25,36.0,903,25.08,677,78,46,57,34,11
1,Aston Villa,28,36.0,812,22.56,574,82,48,56,38,14
2,Bournemouth,29,36.0,964,26.78,735,65,40,65,38,21
3,Brentford,28,36.0,743,20.64,542,70,39,47,34,11
4,Brighton,31,36.0,855,23.75,633,62,52,43,51,14


In [126]:
squad_goal_shot_creation.columns

Index(['Squad', '# Pl', '90s', 'SCA', 'SCA90', 'PassLive', 'PassDead', 'TO',
       'Sh', 'Fld', 'Def'],
      dtype='object')

In [127]:
# Squad goal and shot creation columns to grab: "Squad", "TO" (Successful take-ons that led to a shot attempt), "Fld" (Fouls Drawn)
squad_goal_shot_creation_filtered = squad_goal_shot_creation[['Squad', 'TO', 'Fld']]
squad_goal_shot_creation_filtered.head()

,Squad,TO,Fld
0,Arsenal,46,34
1,Aston Villa,48,38
2,Bournemouth,40,38
3,Brentford,39,34
4,Brighton,52,51


In [128]:
squad_goal_shot_creation_filtered.columns

Index(['Squad', 'TO', 'Fld'], dtype='object')

In [129]:
# Merging all the DataFrames, start with first dataframe
epl_merged = squad_standings_filtered

# Merge other DataFrames one by one
epl_merged = pd.merge(epl_merged, squad_stats_filtered, on="Squad", how="left")
epl_merged.head()

,Squad,W,D,L,GD,Attendance,# Pl,Age,Poss,G+A,CrdY,CrdR,PrgC,PrgP
0,Arsenal,18,14,4,33,60256,25,26.6,56.9,116,67,6,806,1738
1,Aston Villa,18,9,9,7,42070,28,27.8,50.6,97,73,3,678,1254
2,Bournemouth,14,11,11,12,10587,29,25.9,48.3,93,94,2,719,1411
3,Brentford,16,7,13,10,18832,28,26.6,47.7,104,57,1,559,1273
4,Brighton,14,13,9,3,31808,31,25.7,52.1,95,76,3,766,1432


In [130]:
# Merge other DataFrames one by one
epl_merged = pd.merge(epl_merged, squad_shooting_filtered, on="Squad", how="left")
# Merge other DataFrames one by one
epl_merged = pd.merge(epl_merged, squad_passing_filtered, on="Squad", how="left")
# Merge other DataFrames one by one
epl_merged = pd.merge(epl_merged, squad_goal_shot_creation_filtered, on="Squad", how="left")
epl_merged

,Squad,W,D,L,GD,Attendance,# Pl,Age,Poss,G+A,...,Sh,SoT,G/Sh,Dist,Total Cmp%,Short Cmp%,Medium Cmp%,Long Cmp%,TO,Fld
0,Arsenal,18,14,4,33,60256,25,26.6,56.9,116,...,509,167,0.12,15.0,84.3,91.7,89.1,51.3,46,34
1,Aston Villa,18,9,9,7,42070,28,27.8,50.6,97,...,454,151,0.11,16.2,82.7,90.2,88.9,53.1,48,38
2,Bournemouth,14,11,11,12,10587,29,25.9,48.3,93,...,546,183,0.09,16.7,76.5,86.4,84.6,48.2,40,38
3,Brentford,16,7,13,10,18832,28,26.6,47.7,104,...,411,162,0.14,15.0,78.1,86.7,84.8,52.6,39,34
4,Brighton,14,13,9,3,31808,31,25.7,52.1,95,...,479,164,0.11,17.2,82.3,90.5,87.4,54.4,52,51
5,Chelsea,18,9,9,19,39662,29,24.5,57.6,104,...,573,198,0.10,16.7,84.9,91.7,90.6,54.8,57,38
6,Crystal Palace,12,13,11,-2,25081,29,27.0,43.4,78,...,490,160,0.08,16.8,75.5,85.5,83.8,44.9,31,45
7,Everton,9,15,12,-5,38397,26,28.8,40.8,61,...,377,129,0.09,16.9,76.0,86.5,84.3,52.4,40,29
8,Fulham,14,9,13,1,26779,26,28.8,52.5,93,...,487,157,0.10,16.4,82.0,89.7,88.2,55.5,36,23
9,Ipswich Town,4,10,22,-42,29740,32,26.6,40.3,59,...,340,114,0.09,17.8,77.7,88.3,85.3,46.5,47,29


In [133]:
# write this data to a csv file
epl_merged.to_csv("eplMerged.csv")

In [131]:
epl_merged.columns

Index(['Squad', 'W', 'D', 'L', 'GD', 'Attendance', '# Pl', 'Age', 'Poss',
       'G+A', 'CrdY', 'CrdR', 'PrgC', 'PrgP', 'Sh', 'SoT', 'G/Sh', 'Dist',
       'Total Cmp%', 'Short Cmp%', 'Medium Cmp%', 'Long Cmp%', 'TO', 'Fld'],
      dtype='object')